In [1]:
import pandas as pd
import ast
from bs4 import BeautifulSoup
import requests
from textblob import TextBlob

# Limpieza y transformación del dataset de output steam games

In [2]:
# pasamos los archivos a un Dataframe
dfoutput=pd.read_json(r'steam_games.json.gz',lines=True, encoding='MacRoman')

In [3]:
dfoutput.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')

In [268]:
#conservamos las columnas que nos van a servir para nuestros objetivos
dfjuegos=pd.DataFrame(dfoutput[['id','app_name','developer','publisher','release_date','url']].drop_duplicates()[1:])

In [269]:
dfjuegos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 88310 to 120444
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            32133 non-null  float64
 1   app_name      32133 non-null  object 
 2   developer     28836 non-null  object 
 3   publisher     24083 non-null  object 
 4   release_date  30068 non-null  object 
 5   url           32135 non-null  object 
dtypes: float64(1), object(5)
memory usage: 1.7+ MB


In [270]:
#vemos de primera instancia si hay duplicados
dfjuegos.duplicated().sum()

0

In [271]:
#buscamos nulos en el dataframe para tratarlos
dfjuegos.isnull().sum()

id                 2
app_name           2
developer       3299
publisher       8052
release_date    2067
url                0
dtype: int64

In [273]:
#vemos dos nulo en id
dfjuegos[dfjuegos.id.isnull()]

,id,app_name,developer,publisher,release_date,url
88384,NaN,None,None,None,None,http://store.steampowered.com/
119271,NaN,Batman: Arkham City - Game of the Year Edition,"Rocksteady Studios,Feral Interactive (Mac)","Warner Bros. Interactive Entertainment, Feral ...",2012-09-07,http://store.steampowered.com/app/200260


In [274]:
#verficamos si esta duplicado y efectivamente parece que lo esta, comprobamos quitando la columna id
print(dfjuegos[dfjuegos.app_name==dfjuegos.app_name[dfjuegos.id.isnull()].iloc[0]])
dfjuegos.drop(index=dfjuegos.loc[dfjuegos.id.isnull()].index[0],inplace=True)
dfjuegos.drop(index=dfjuegos.loc[dfjuegos.id.isnull()].index[0],inplace=True)
dfjuegos.isnull().sum()

Empty DataFrame
Columns: [id, app_name, developer, publisher, release_date, url]
Index: []


id                 0
app_name           1
developer       3298
publisher       8051
release_date    2066
url                0
dtype: int64

In [275]:
#cambiamos el tipo de dato de la columna id a int
dfjuegos['id']=dfjuegos.id.astype(int)
dfjuegos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32133 entries, 88310 to 120444
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            32133 non-null  int32 
 1   app_name      32132 non-null  object
 2   developer     28835 non-null  object
 3   publisher     24082 non-null  object
 4   release_date  30067 non-null  object
 5   url           32133 non-null  object
dtypes: int32(1), object(5)
memory usage: 1.6+ MB


In [276]:
#vemos 1 nulo en app_name
dfjuegos[dfjuegos.app_name.isnull()]

,id,app_name,developer,publisher,release_date,url
90890,317160,None,None,None,2014-08-26,http://store.steampowered.com/app/317160/_/


In [277]:
# el nulo en app_name lo sacamos del url del juego
url=dfoutput.url[dfoutput.id==dfjuegos.id[dfjuegos.app_name.isnull()].iloc[0]].iloc[0]
url

'http://store.steampowered.com/app/317160/_/'

In [278]:
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
titulo=soup.head.title
titulo=titulo.text
titulo=titulo.removesuffix(' on Steam')
titulo

'Save 10% on Duet'

In [279]:
#colocamos el nombre de la aplicacion faltante obtenido de la pagina web y verificamos que la columna app_name ya no tiene nulos
dfjuegos.loc[dfjuegos.app_name.isnull(),'app_name']=titulo
dfjuegos.isnull().sum()

id                 0
app_name           0
developer       3298
publisher       8051
release_date    2066
url                0
dtype: int64

In [280]:
# sustituimos valores nulos de la columna desarrolladores con la columna de publihser, solo en las filas donde no hya nulos en publisher
dfjuegos.loc[(dfjuegos.developer.isnull())&(~dfjuegos.publisher.isnull()),'developer']=dfjuegos.publisher
#sustituimos lo valores nulos restantes con 'N/D'
dfjuegos.loc[(dfjuegos.developer.isnull()),'developer']='N/D'
dfjuegos.isnull().sum()

id                 0
app_name           0
developer          0
publisher       8051
release_date    2066
url                0
dtype: int64

In [281]:
#quitamos la columna publisher, ya que solo nos interesa la de desarrollador
dfjuegos.drop(columns='publisher',inplace=True)
dfjuegos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32133 entries, 88310 to 120444
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            32133 non-null  int32 
 1   app_name      32133 non-null  object
 2   developer     32133 non-null  object
 3   release_date  30067 non-null  object
 4   url           32133 non-null  object
dtypes: int32(1), object(4)
memory usage: 2.4+ MB


In [282]:
#sacamos el año de lanzamiento de la columna release_date y quitamos la columna release_date
dfjuegos['Año_lanzamiento']=dfjuegos.release_date.str.split('-',expand=True)[0]


In [283]:
dfjuegos.head()

,id,app_name,developer,release_date,url,Año_lanzamiento
88310,761140,Lost Summoner Kitty,Kotoshiro,2018-01-04,http://store.steampowered.com/app/761140/Lost_...,2018
88311,643980,Ironbound,Secret Level SRL,2018-01-04,http://store.steampowered.com/app/643980/Ironb...,2018
88312,670290,Real Pool 3D - Poolians,Poolians.com,2017-07-24,http://store.steampowered.com/app/670290/Real_...,2017
88313,767400,弹炸人2222,彼岸领域,2017-12-07,http://store.steampowered.com/app/767400/2222/,2017
88314,773570,Log Challenge,N/D,None,http://store.steampowered.com/app/773570/Log_C...,None


In [284]:
#renombramos los nombres de las columnas
dfjuegos.rename(columns={'Año lanzamiento':'Año_lanzamiento','app_name':'Nombre','developer':'Desarrollador','id':'Id_juego'},inplace=True)

In [285]:
#hacemos una lista de los años que hay en la columna Año_de_lanzamiento
años=[]
for a in dfjuegos.Año_lanzamiento.sort_values().unique():
    try:
        int(a)
    except:
        continue
    else:
        años.append(a)
años[0:5]

['1970', '1975', '1980', '1981', '1982']

In [286]:
#cambiamos las filas que tienen el año en una parte de su cadena a contener solo el dato del año
for a in años:
    dfjuegos.loc[dfjuegos.Año_lanzamiento.str.contains(a,na=False),'Año_lanzamiento']=a

In [ ]:
#haciendo webscrapping, con los datos que tenemos podemos sacar el año de lanzamiento faltante
url=dfjuegos[['url']][dfjuegos.release_date.isnull()].iloc[0][0]
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
text=soup.get_text()
print(soup.prettify())
print(text)
div_subtitulo = soup.find('div', class_='date')
div_subtitulo.text.strip("<div class='date'")


In [299]:
#haciendo webscrapping, con los datos que tenemos buscamos el año para cada juego faltante
for a in dfjuegos.url[dfjuegos.Año_lanzamiento.isnull()]:
    url=a
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    div_subtitulo = soup.find('div', class_='date')
    try:
        dfjuegos.loc[dfjuegos.url==a,'Año_lanzamiento']=str(div_subtitulo.text.strip("<div class='date'").split()[-1])
    except:
        continue


In [ ]:
#vemos que hay valores nulos aun que no se pudieron recuperar de la web de steam
dfjuegos.isnull().sum()

In [303]:
#rellenamos nulos con el valor'N/D'
dfjuegos.loc[dfjuegos.Año_lanzamiento.isnull(),'Año_lanzamiento']='N/D'
dfjuegos.isnull().sum()

Id_juego              0
Nombre                0
Desarrollador         0
release_date       2066
url                   0
Año_lanzamiento       0
dtype: int64

In [304]:
dfjuegos.Año_lanzamiento.sort_values().unique()

array(['"""Soon"""',
       '0̵1̴0̵0̶1̷0̶0̵0̴ ̴0̶0̶1̶1̶0̷0̶1̵1̴ ̸0̶0̶1̶1̵0̶1̷0̴0̵ ̴0̶1̷0̸1̵0̷0̴1̶0̴ ̴0̷0̴1̷1̶0̶1̵1̷1̵ ̵',
       '14 July', '1970', '1975', '1980', '1981', '1982', '1983', '1984',
       '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992',
       '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019', '2020', '2021', '2022', '2023',
       'Beta测试已开启', "C'est bientôt...                    (ou pas)",
       'Coming Soon', 'Coming Soon!', 'Coming Soon/Próximamente',
       'Coming soon', 'Coming soon..', 'Comming Soon',
       'Datachunks conflicted. Be vigiliant.', 'Demo coming soon.',
       'Demo is available now!', 'Early Access Starting Soon!',
       'Early Access soon', 'N/D', 'Not yet available',
       'Play Beta in demo!', 'Please wait warmly', 'Release Date TBA',
       'S

In [308]:
# se hace una lista de los datos que asiguen pendientes en la columna año de lanzamiento, y haciendo investigacion se encuentra que los juegos fueron lanzados alrededor del 2018 en adelante
datossinaño=[]
for a in dfjuegos.Año_lanzamiento.sort_values().unique():
    if a not in años  and a!='N/D':
        datossinaño.append(a)
datossinaño

['"""Soon"""',
 '0̵1̴0̵0̶1̷0̶0̵0̴ ̴0̶0̶1̶1̶0̷0̶1̵1̴ ̸0̶0̶1̶1̵0̶1̷0̴0̵ ̴0̶1̷0̸1̵0̷0̴1̶0̴ ̴0̷0̴1̷1̶0̶1̵1̷1̵ ̵',
 '14 July',
 '2020',
 '2022',
 '2023',
 'Beta测试已开启',
 "C'est bientôt...                    (ou pas)",
 'Coming Soon',
 'Coming Soon!',
 'Coming Soon/Próximamente',
 'Coming soon',
 'Coming soon..',
 'Comming Soon',
 'Datachunks conflicted. Be vigiliant.',
 'Demo coming soon.',
 'Demo is available now!',
 'Early Access Starting Soon!',
 'Early Access soon',
 'Not yet available',
 'Play Beta in demo!',
 'Please wait warmly',
 'Release Date TBA',
 'SOON',
 'SOON™',
 'Soon',
 'Soon..',
 'Soon™',
 'TBA',
 'TBD',
 'To Be Announced',
 'To be Announced',
 'To be announced',
 'When it is finished',
 'When it is ready',
 "When it's done",
 "When it's done!",
 'coming soon',
 'early access',
 'soon',
 '预热群52756441']

In [309]:
#
dfjuegos.url[dfjuegos.Año_lanzamiento.isin(datossinaño)]

88320     http://store.steampowered.com/app/768570/Uncan...
88352     http://store.steampowered.com/app/662570/Unite...
88354     http://store.steampowered.com/app/714020/Idle_...
88355     http://store.steampowered.com/app/541930/Panop...
88363     http://store.steampowered.com/app/374970/Golf_...
                                ...                        
120302    http://store.steampowered.com/app/391480/Mutat...
120310    http://store.steampowered.com/app/689820/Spy_o...
120318       http://store.steampowered.com/app/519860/DUSK/
120397    http://store.steampowered.com/app/755830/Lonel...
120398    http://store.steampowered.com/app/708070/RECHA...
Name: url, Length: 190, dtype: object

In [310]:
#haciendo webscrapping, con los datos que tenemos sin dato exacto del año los buscamos 
for a in dfjuegos.url[dfjuegos.Año_lanzamiento.isin(datossinaño)]:
    url=a
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    div_subtitulo = soup.find('div', class_='date')
    try:
        dfjuegos.loc[dfjuegos.url==a,'Año_lanzamiento']=str(div_subtitulo.text.strip("<div class='date'").split()[-1])
    except:
        continue


In [312]:
# se vuelve a hacer una lista de los añosque siguen pendientes en la columna año de lanzamiento al igual que una con los años en formato correcto

años=[]
for a in dfjuegos.Año_lanzamiento.sort_values().unique():
    try:
        int(a)
    except:
        continue
    else:
        años.append(a)
años[0:5]
datossinaño=[]
for a in dfjuegos.Año_lanzamiento.sort_values().unique():
    if a not in años  and a!='N/D':
        datossinaño.append(a)
datossinaño

['0̵1̴0̵0̶1̷0̶0̵0̴ ̴0̶0̶1̶1̶0̷0̶1̵1̴ ̸0̶0̶1̶1̵0̶1̷0̴0̵ ̴0̶1̷0̸1̵0̷0̴1̶0̴ ̴0̷0̴1̷1̶0̶1̵1̷1̵ ̵',
 'Coming Soon',
 'SOON',
 'Soon',
 'TBD',
 'announ',
 'soon']

In [314]:
# estos datos sin años represnetan solo una pequeña parte de nuestro dataset de juegos, se les imputara el valor de N/D,
#ya que represntan una peueña parte de nuestro dataset de juegos
dfjuegos.loc[dfjuegos.Año_lanzamiento.isin(datossinaño),'Año_lanzamiento']='N/D'

In [315]:
#hacemos una lista de los años que hay en la columna Año_de_lanzamiento y comprobamos que ya estan normalizados todos los datos del año de lanzamiento
dfjuegos.Año_lanzamiento.sort_values().unique()


array(['1970', '1975', '1980', '1981', '1982', '1983', '1984', '1985',
       '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993',
       '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001',
       '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019', '2020', '2021', '2022', '2023', '2024', 'N/D'],
      dtype=object)

In [316]:
# unimos los generos al dataframe de juegos
dfjuegos=dfjuegos.join(dfoutput[['genres','tags']],how='inner')


In [317]:
dfjuegos.head()

,Id_juego,Nombre,Desarrollador,release_date,url,Año_lanzamiento,genres,tags
88310,761140,Lost Summoner Kitty,Kotoshiro,2018-01-04,http://store.steampowered.com/app/761140/Lost_...,2018,"[Action, Casual, Indie, Simulation, Strategy]","[Strategy, Action, Indie, Casual, Simulation]"
88311,643980,Ironbound,Secret Level SRL,2018-01-04,http://store.steampowered.com/app/643980/Ironb...,2018,"[Free to Play, Indie, RPG, Strategy]","[Free to Play, Strategy, Indie, RPG, Card Game..."
88312,670290,Real Pool 3D - Poolians,Poolians.com,2017-07-24,http://store.steampowered.com/app/670290/Real_...,2017,"[Casual, Free to Play, Indie, Simulation, Sports]","[Free to Play, Simulation, Sports, Casual, Ind..."
88313,767400,弹炸人2222,彼岸领域,2017-12-07,http://store.steampowered.com/app/767400/2222/,2017,"[Action, Adventure, Casual]","[Action, Adventure, Casual]"
88314,773570,Log Challenge,N/D,None,http://store.steampowered.com/app/773570/Log_C...,2018,None,"[Action, Indie, Casual, Sports]"


In [318]:
#vemos cuantos generos sin dato hay
dfjuegos.genres.isnull().sum()

3282

In [388]:
dfjuegos[dfjuegos.genres=='Title: Robotpencil Presents: Character Design - Horror Genre']

,Id_juego,Nombre,Desarrollador,release_date,url,Año_lanzamiento,genres,tags
93094,413910,Robotpencil Presents: Character Design - Horro...,N/D,2015-11-12,http://store.steampowered.com/app/413910/Robot...,2015,Title: Robotpencil Presents: Character Design ...,"[Design & Illustration, Tutorial]"


In [392]:
dfjuegos[dfjuegos.genres.isnull()]

,Id_juego,Nombre,Desarrollador,release_date,url,Año_lanzamiento,genres,tags
88336,777910,Robotpencil Presents: Understanding 3D for Con...,N/D,2018-01-03,http://store.steampowered.com/app/777910/Robot...,2018,None,"[Design & Illustration, Tutorial]"
88361,769350,Medicalholodeck Personal Version,N/D,None,http://store.steampowered.com/app/769350/Medic...,N/D,None,[Education]
88459,12580,Dr. Daisy Pet Vet,Zemnott,2008-01-08,http://store.steampowered.com/app/12580/Dr_Dai...,2008,None,[Casual]
88461,12570,Hot Dish,Zemnott,2008-01-12,http://store.steampowered.com/app/12570/Hot_Dish/,2008,None,[Casual]
88562,22340,Call of Cthulhu®: Dark Corners of the Earth,Headfirst Productions,2006-04-26,http://store.steampowered.com/app/22340/Call_o...,2006,None,"[Horror, Lovecraftian, Survival Horror, First-..."
...,...,...,...,...,...,...,...,...
120297,775660,Robotpencil Presents: Start with Color,N/D,2017-12-28,http://store.steampowered.com/app/775660/Robot...,2017,None,"[Design & Illustration, Tutorial]"
120428,775640,Robotpencil Presents: Exercise: Brushwork,N/D,2018-01-03,http://store.steampowered.com/app/775640/Robot...,2018,None,"[Design & Illustration, Tutorial]"
120429,777930,Robotpencil Presents: Creative Composition,N/D,2018-01-03,http://store.steampowered.com/app/777930/Robot...,2018,None,"[Design & Illustration, Tutorial]"
120430,775370,The Gamble House,N/D,2016-11-19,http://store.steampowered.com/app/775370/The_G...,2016,None,[Movie]


In [380]:
#hacemos webscrapping con los generos que no sabemos
for a in dfjuegos['url'][dfjuegos.genres.isnull()]:
    url=a
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    div_subtitulo = soup.find('div', class_='block_content_inner')
    try:
        for b in div_subtitulo.text.split('\n'):
            if 'Genre' in b:
                b=b.replace('Genre: ','')
                dfjuegos.loc[dfjuegos.url==a,'genres']=b
                print(f'nulos restantes:{dfjuegos.genres.isnull().sum()}')
                break
    except:
        continue

nulos restantes:3281
nulos restantes:3280
nulos restantes:3279
nulos restantes:3278
nulos restantes:3277
nulos restantes:3276
nulos restantes:3275
nulos restantes:3274
nulos restantes:3273
nulos restantes:3272
nulos restantes:3271
nulos restantes:3270
nulos restantes:3269
nulos restantes:3268
nulos restantes:3267
nulos restantes:3266
nulos restantes:3265
nulos restantes:3264
nulos restantes:3263
nulos restantes:3262
nulos restantes:3261
nulos restantes:3260
nulos restantes:3259
nulos restantes:3258
nulos restantes:3257
nulos restantes:3256
nulos restantes:3255
nulos restantes:3254
nulos restantes:3253
nulos restantes:3252
nulos restantes:3251
nulos restantes:3250
nulos restantes:3249
nulos restantes:3248
nulos restantes:3247
nulos restantes:3246
nulos restantes:3245
nulos restantes:3244
nulos restantes:3243
nulos restantes:3242
nulos restantes:3241
nulos restantes:3240
nulos restantes:3239
nulos restantes:3238
nulos restantes:3237
nulos restantes:3236
nulos restantes:3235
nulos restant

In [381]:
#vemos cuantos generos sin dato hay
dfjuegos.genres.isnull().sum()

1289

In [115]:
# creamos una lista con los generos existentes
generos=[]
for a in dfjuegos.genres.dropna().drop_duplicates():
    if type(a)==list:
        for b in a:
            if b not in generos:
                generos.append(b)
    else:
        for c in a.split(', '):
            c=c.strip("'[] ") 
            if c not in generos:
                generos.append(c)
generos=sorted(generos)
print(generos)


['Accounting', 'Action', 'Adventure', 'Animation & Modeling', 'Animation &amp; Modeling', 'Audio Production', 'Casual', 'Design & Illustration', 'Design &amp; Illustration', 'Early Access', 'Education', 'Free to Play', 'Game Development', 'Indie', 'Massively Multiplayer', 'Photo Editing', 'RPG', 'Racing', 'Simulation', 'Software Training', 'Sports', 'Strategy', 'Title: Robotpencil Presents: Character Design - Horror Genre', 'Utilities', 'Video Production', 'Web Publishing']


In [116]:
len(generos)

26

In [117]:
# Agregamos 'N/D' por los generos que desconozcamos
generos.append('N/D')
generos

['Accounting',
 'Action',
 'Adventure',
 'Animation & Modeling',
 'Animation &amp; Modeling',
 'Audio Production',
 'Casual',
 'Design & Illustration',
 'Design &amp; Illustration',
 'Early Access',
 'Education',
 'Free to Play',
 'Game Development',
 'Indie',
 'Massively Multiplayer',
 'Photo Editing',
 'RPG',
 'Racing',
 'Simulation',
 'Software Training',
 'Sports',
 'Strategy',
 'Title: Robotpencil Presents: Character Design - Horror Genre',
 'Utilities',
 'Video Production',
 'Web Publishing',
 'N/D']

In [118]:
len(generos)

27

In [32]:

dfjuegos=pd.merge(dfjuegos,dfoutput[['id','tags']],left_on='Id_juego',right_on='id',how='inner')

In [129]:
# En la columna tags tenemos algunos valores de los generos que faltan en la columna generos
#los tomareos de ahi haciendonuna lista solo con los generos que tenenos ya en la columna generos
def genero(lista,lista2):
    b=[]
    for a in lista:
        if a in lista2:
            b.append(a)
    if len(b)==0:
        b.append('N/D')           
    return b
#rellenamos con 'N/D' para poder identificar los generos donde no sabemos que valor imputar
dfjuegos.loc[dfjuegos.tags.isna(),'tags']='N/D'
dfjuegos.loc[(dfjuegos.genres.isnull())&(~dfjuegos.tags.isnull()),'tags']=dfjuegos.loc[(dfjuegos.genres.isnull())&(~dfjuegos.tags.isnull()),'tags'].apply(lambda x: genero(x,generos))
dfjuegos.loc[(dfjuegos.genres.isnull())&(~dfjuegos.tags.isnull()),'genres']=dfjuegos.loc[(dfjuegos.genres.isnull())&(~dfjuegos.tags.isnull()),'tags']
dfjuegos.loc[(dfjuegos.genres.isnull())&(~dfjuegos.tags.isnull()),['genres','tags']]
            

,genres,tags


In [133]:
dfjuegos.columns

Index(['Id_juego', 'Nombre', 'Desarrollador', 'release_date', 'url',
       'Año_lanzamiento', 'genres', 'id'],
      dtype='object')

In [135]:
# eliminamos la columna tags, release_date y url ya que ya no la utlizaremos, verificamos que ya no hya valores nulos y cambiamos nombre de las columna genres a Género
dfjuegos.drop(columns=['release_date', 'url','id','tags'],inplace=True)
print(dfjuegos.isnull().sum())
dfjuegos.rename(columns={'genres':'Género'},inplace=True)
print(dfjuegos.columns)


Id_juego           0
Nombre             0
Desarrollador      0
Año_lanzamiento    0
Género             0
dtype: int64
Index(['Id_juego', 'Nombre', 'Desarrollador', 'Año_lanzamiento', 'Género'], dtype='object')


In [9]:
# en dos géneros vienen con la expresion &amp; y se remplaza por &
dfjuegos.loc[dfjuegos.Género.str.contains('amp'),'Género']=dfjuegos.Género[dfjuegos.Género.str.contains('amp')].str.replace('&amp;','&')

In [34]:
# se cambia un genero que quedo mal al hacer weebscrapping automatico, y se le pone el tag
id_gen_cambiar=dfjuegos.loc[dfjuegos['Género']=='Title: Robotpencil Presents: Character Design - Horror Genre','Id_juego'].iloc[0]
print(dfoutput.tags[dfoutput['id']==id_gen_cambiar].iloc[0][0])
dfjuegos.loc[dfjuegos['Género']=='Title: Robotpencil Presents: Character Design - Horror Genre','Género']=dfoutput.tags[dfoutput['id']==id_gen_cambiar].iloc[0][0]

Design & Illustration


In [35]:
#guardamos en csv para usar despues en nuestras funciones
dfjuegos.to_csv('Juegos_steam.csv',index=False)

# ETl de dataset users items

In [5]:
#abirmos el rchivo de items en un dataframe
with open('australian_users_items.json',encoding='MacRoman') as archivo_json:
    contenido_json = archivo_json.readlines()
lisitems=[eval(line.strip())for line in contenido_json]
dfitems=pd.DataFrame(lisitems)
dfitems

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [6]:
# Se desanida la columna de items y se pasa a un Dataframe el cual contiene la informacion de cada juego jugado por el usuario, como horas jugadas y el id del juego
#dfitems=dfitems.explode('items')
# Se desanida la columna de items y se pasa a un Dataframe el cual contiene la informacion de cada juego jugado por el usuario, como horas jugadas y el id del juego
lista_items=[] 
lista_items2=[]
for a in dfitems['items']:
    lista_items.append(ast.literal_eval(str(a)))
for a in range(len(lista_items)):
    for b in range(len(lista_items[a])):
        lista_items[a][b]['user_id']=lisitems[a]['user_id']
        lista_items2.append(lista_items[a][b])
        

In [7]:
dfitems=pd.DataFrame(lista_items2)

In [8]:
#observamos que hay filas duplicadas en los items
dfitems.duplicated().sum()

59104

In [9]:
#se procede a eliminar duplicados del dataframe de items y verificamos que se eliminaron
dfitems.drop_duplicates(inplace=True,ignore_index=True)
dfitems.duplicated().sum()

0

In [10]:
#cambiamos el tipo de dato a int de la columna de ids del item
dfitems.item_id=dfitems.item_id.astype(int)

In [11]:
dfitems.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094105 entries, 0 to 5094104
Data columns (total 5 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           int32 
 1   item_name         object
 2   playtime_forever  int64 
 3   playtime_2weeks   int64 
 4   user_id           object
dtypes: int32(1), int64(2), object(2)
memory usage: 174.9+ MB


In [12]:
#verificamos que no hya datos nulos en nuestro dataset
dfitems.isnull().sum()

item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
user_id             0
dtype: int64

In [13]:
#cambiamos nombres de las columnas 
columnas_items={'item_id':'Id_juego','item_name':'Nombre_juego','playtime_forever':'Horas_jugadas','user_id':'Id_usuario'}
dfitems.rename(columns=columnas_items,inplace=True)
dfitems.columns

Index(['Id_juego', 'Nombre_juego', 'Horas_jugadas', 'playtime_2weeks',
       'Id_usuario'],
      dtype='object')

In [14]:
#la columna Horas juagadas viene en minutos, por lo que se procede a hacer la conversión a horas para que se pueda llevar a cabo las consignas
dfitems.Horas_jugadas=round(dfitems.Horas_jugadas/60,2)

In [15]:
#quitamos la columna playtime_2weeks, no se usara para nuestras funciones
dfitems.drop(columns='playtime_2weeks',inplace=True)

In [16]:
#visualizamos nuestro dataframe para guardar en un csv
dfitems.head()

,Id_juego,Nombre_juego,Horas_jugadas,Id_usuario
0,10,Counter-Strike,0.10,76561197970982479
1,20,Team Fortress Classic,0.00,76561197970982479
2,30,Day of Defeat,0.12,76561197970982479
3,40,Deathmatch Classic,0.00,76561197970982479
4,50,Half-Life: Opposing Force,0.00,76561197970982479


In [17]:
#verificamos nuevamente prar corroborar duplicados y nulos
print('duplicados: ', dfitems.duplicated().sum())
print('nulos: ', dfitems.isnull().sum().sum())

duplicados:  13
nulos:  0


In [18]:
#eliminamos duplicaods nuevamente
dfitems.drop_duplicates(inplace=True)
print('duplicados: ', dfitems.duplicated().sum())

duplicados:  0


In [19]:
# Se guarda en un csv el dataframe con los items desanidados y el id de usuario
dfitems.to_csv('items_por_usuario.csv',index=False)

In [19]:
#se guarda en un parquet para poder disponer subir al repositorio en github
dfitems.to_parquet('items_por_usuario.parquet')

# ETL reviews

In [150]:
# abrimos el archivo reviews
with open('australian_user_reviews.json', encoding='utf-8') as archivo_json:
    contenido_json = archivo_json.readlines()
lisreviews=list(map(ast.literal_eval,contenido_json))
dfreviews=pd.DataFrame(lisreviews)
dfreviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [151]:
# se desanida la columna items
dfreviews.drop(columns='user_url', inplace=True)
dfreviews=dfreviews.explode('reviews')
dfreviews.dropna(inplace=True)
dfreviews2=pd.DataFrame(dfreviews.reviews.tolist())
dfreviews2.head()

,funny,posted,last_edited,item_id,helpful,recommend,review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [152]:
#conservamos las columnas item_id, recommend, review, posted
dfreviews.drop(columns='reviews', inplace=True)
dfreviews[['item_id','recommend','review','posted']]=dfreviews2[['item_id','recommend','review','posted']]
dfreviews.head()

,user_id,item_id,recommend,review,posted
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,"Posted November 5, 2011."
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,"Posted November 5, 2011."
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,"Posted November 5, 2011."
1,js41637,22200,True,It's unique and worth a playthrough.,"Posted July 15, 2011."
1,js41637,22200,True,It's unique and worth a playthrough.,"Posted July 15, 2011."


In [153]:
#verificamos duplicaods, y los booramos
print(f'valores duplicados: {dfreviews.duplicated().sum()}')
dfreviews.drop_duplicates(inplace=True)
print(f'verificando si quedan valores duplicados: {dfreviews.duplicated().sum()}')

valores duplicados: 33534
verificando si quedan valores duplicados: 0


In [154]:
#buscamos valores nulos del dataframe y corroboramos que no existen
dfreviews.isnull().sum()

user_id      0
item_id      0
recommend    0
review       0
posted       0
dtype: int64

In [155]:
#quitamos la palabra posted, las comas y puntos
dfreviews.posted=dfreviews.posted.str.removeprefix('Posted ').str.replace(',','').str.replace('.','')

In [156]:
#sacamos el año de la columna posted
dfreviews['Año_review']=dfreviews.posted.str.split(expand=True)[2]
dfreviews.head()

,user_id,item_id,recommend,review,posted,Año_review
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,November 5 2011,2011
1,js41637,22200,True,It's unique and worth a playthrough.,July 15 2011,2011
2,evcentric,43110,True,Great atmosphere. The gunplay can be a bit chu...,April 21 2011,2011
3,doctr,251610,True,I know what you think when you see this title ...,June 24 2014,2014
4,maplemage,227300,True,For a simple (it's actually not all that simpl...,September 8 2013,2013


In [157]:
# En la platfroma Steam, los reviews que no tienen año indican que fueron realizados en el año en curso
#lo que nos indica que el año mas grande en la lista de años unicos es el 2015, el año en curso es el 2016
print(f'valores nulos en la columna Año_review: {dfreviews.Año_review.isnull().sum()}')
print(f'ulitmo año especificado en los datos: {dfreviews.Año_review.sort_values(ascending=False).unique()[0]}')
#procedemos a relllenar los años faltantes con 2016
dfreviews.Año_review.fillna('2016',inplace=True)
#verificamos que ya se cambio el dato y que no hay nulos
print(f'valores nulos en la columna Año_review despues de cambiar nulos: {dfreviews.Año_review.isnull().sum()}')
print(f'ulitmo año especificado en los datos: {dfreviews.Año_review.sort_values(ascending=False).unique()[0]}')

valores nulos en la columna Año_review: 5427
ulitmo año especificado en los datos: 2015
valores nulos en la columna Año_review despues de cambiar nulos: 0
ulitmo año especificado en los datos: 2016


In [158]:
#eliminamos la columna posted
dfreviews.drop(columns='posted',inplace=True)

analsis de sentimientos con texblob

In [159]:
def get_sentiment_label(text):
    analysis = TextBlob(text)
    polarity = analysis.sentiment.polarity
    
    if polarity < -0.4:
        return 0  # Malo
    elif polarity > 0.1:
        return 2  # Positivo
    else:
        return 1  # Neutral

# Aplicar la función a la columna 'review' y crear la nueva columna 'sentiment_analysis'
dfreviews['sentiment_analysis'] = dfreviews['review'].apply(get_sentiment_label)

# Mostrar las primeras filas para verificar los cambios
dfreviews.head()

,user_id,item_id,recommend,review,Año_review,sentiment_analysis
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011,2
1,js41637,22200,True,It's unique and worth a playthrough.,2011,2
2,evcentric,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011,1
3,doctr,251610,True,I know what you think when you see this title ...,2014,2
4,maplemage,227300,True,For a simple (it's actually not all that simpl...,2013,1


In [160]:
#verificamos que las reviews donde no se escribió nada tengan 1 como valor como esta indicado en la consigna
dfreviews.sentiment_analysis[(dfreviews.review=='')|(dfreviews.review==' ')].value_counts()

sentiment_analysis
1    17
Name: count, dtype: int64

In [161]:
dfreviews.tail()

,user_id,item_id,recommend,review,Año_review,sentiment_analysis
25794,76561198306599751,226700,False,"This is terrible haha, even if it is on sale f...",2014,1
25795,Ghoustik,304930,True,Is k,2014,1
25796,76561198310819422,208730,False,This game made my thighs moist. I prefer them ...,2014,1
25797,76561198312638244,230410,True,definitely the best action game out there in t...,2014,2
25798,LydiaMorley,440,True,it boss,2013,1


In [162]:
# cambiamos el tipo de dato de la columna item_id y de Año_review
dfreviews.item_id=dfreviews.item_id.astype(int)
dfreviews.Año_review= pd.to_datetime(dfreviews.Año_review, format='%Y')
dfreviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25771 entries, 0 to 25798
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   user_id             25771 non-null  object        
 1   item_id             25771 non-null  int32         
 2   recommend           25771 non-null  bool          
 3   review              25771 non-null  object        
 4   Año_review          25771 non-null  datetime64[ns]
 5   sentiment_analysis  25771 non-null  int64         
dtypes: bool(1), datetime64[ns](1), int32(1), int64(1), object(2)
memory usage: 1.1+ MB


In [163]:
#cambiamos el nombre de las columnas
dfreviews.rename(columns={'item_id':'Id_juego','recommend':'Recommend','user_id':'Id_usuario'},inplace=True)
dfreviews.columns

Index(['Id_usuario', 'Id_juego', 'Recommend', 'review', 'Año_review',
       'sentiment_analysis'],
      dtype='object')

In [164]:
#quitamos la columna reviews
dfreviews.drop(columns='review',inplace=True)
dfreviews.columns

Index(['Id_usuario', 'Id_juego', 'Recommend', 'Año_review',
       'sentiment_analysis'],
      dtype='object')

In [165]:
#guardamos en un csv
dfreviews.to_csv('revies_por_usuario.csv',index=False)